In [14]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import re

rootURL = 'https://www.pro-football-reference.com/'
playerURL_list = []
player_list = []

ppr = 1

# HTML cleaner
def _strip_html(text):
    tag_re = re.compile(r'<[^>]+>')
    return tag_re.sub('', str(text))

url = 'https://www.pro-football-reference.com/years/2020/scrimmage.htm'
## Scrape for RB stats
res = requests.get(url, verify=False)
soup = BeautifulSoup(res.text,features="html.parser")

parsed = soup.findAll(
    'table', {
        'id': 'receiving_and_rushing'
    }
)

rows = parsed[0].findAll('td')

column_len = 30

grouped_rows = [
    rows[i:i+column_len] for i in range(0, len(rows), column_len)
]


for i in grouped_rows:
    fullSTR = str(i)
    trimmed = re.search('href="(.*).htm">',fullSTR)
    playerURL = trimmed.group(1)
    playerURL = rootURL + playerURL + '.htm'
    playerURL_list.append(playerURL)

c:\users\00141396\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.pro-football-reference.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
import pandas as pd

In [3]:
url = playerURL_list[0]

player_data = pd.read_html(url, header=1)

In [4]:
player_info = player_data[0].ffill().bfill().iloc[:-1, :]
player_info = player_info.apply(lambda x: x.astype(str).str.upper())

In [5]:
player_info.columns

Index(['Year', 'Age', 'Tm', 'Pos', 'No.', 'G', 'GS', 'Rush', 'Yds', 'TD', '1D',
       'Lng', 'Y/A', 'Y/G', 'A/G', 'Tgt', 'Rec', 'Yds.1', 'Y/R', 'TD.1',
       '1D.1', 'Lng.1', 'R/G', 'Y/G.1', 'Ctch%', 'Y/Tgt', 'Touch', 'Y/Tch',
       'YScm', 'RRTD', 'Fmb', 'AV'],
      dtype='object')

In [6]:
player_info['Year'] = player_info['Year'].str.replace('+', '').str.replace('*', '')
player_info['Ctch%'] = player_info['Ctch%'].str.replace('%', '')

<ipython-input-6-fa9f53a27ce0>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  player_info['Year'] = player_info['Year'].str.replace('+', '').str.replace('*', '')


In [7]:
column_names = ['Year', 'Age', 'Team', 'Pos', 'No', 'Games', 'GameStarted', 'RushAtts', 'RushYds', 'RushTD', 'Rush1D', 'RushLong', 'YdsRush', 'YdsGame', 'RushAttGame', 'RecTarget', 'Receptions', 'RecYds', 'YdsRec', 'RecTD', 'Rec1D', 'RecLong', 'RecGame', 'RecYdsGame', 'CatchPerc', 'YdsTarget', 'Touches', 'YdsTouch', 'ScrimYds', 'RushRecTD', 'Fumbles', 'ApproxValue']

In [8]:
player_info.columns = column_names

In [9]:
player_info

,Year,Age,Team,Pos,No,Games,GameStarted,RushAtts,RushYds,RushTD,...,RecGame,RecYdsGame,CatchPerc,YdsTarget,Touches,YdsTouch,ScrimYds,RushRecTD,Fumbles,ApproxValue
0,2016,22,TEN,RB,22.0,15,2,110,490,5,...,0.9,9.1,86.7,9.1,123,5.1,627,5,0,4
1,2017,23,TEN,RB,22.0,16,2,176,744,5,...,0.7,8.5,64.7,8.0,187,4.7,880,6,1,6
2,2018,24,TEN,RB,22.0,16,12,215,1059,12,...,0.9,6.2,83.3,5.5,230,5.0,1158,12,1,7
3,2019,25,TEN,RB,22.0,15,15,303,1540,16,...,1.2,13.7,75.0,8.6,321,5.4,1746,18,5,13
4,2020,26,TEN,RB,22.0,16,16,378,2027,17,...,1.2,7.1,61.3,3.7,397,5.4,2141,17,3,17


In [10]:
# Scoring:
# 0.1 / rec yd
# 1 / reception
# 6 / rec td
# 0.1 / rush yd
# 6 / rush td

In [11]:
player_info[player_info.columns[5:]] = player_info[player_info.columns[5:]].astype(float)

In [15]:
player_info['TotScore'] = (player_info['RushRecTD'] * 6) + (player_info['ScrimYds'] * 0.1) + (player_info['Receptions'] * ppr)

In [16]:
player_info

,Year,Age,Team,Pos,No,Games,GameStarted,RushAtts,RushYds,RushTD,...,RecYdsGame,CatchPerc,YdsTarget,Touches,YdsTouch,ScrimYds,RushRecTD,Fumbles,ApproxValue,TotScore
0,2016,22,TEN,RB,22.0,15.0,2.0,110.0,490.0,5.0,...,9.1,86.7,9.1,123.0,5.1,627.0,5.0,0.0,4.0,105.7
1,2017,23,TEN,RB,22.0,16.0,2.0,176.0,744.0,5.0,...,8.5,64.7,8.0,187.0,4.7,880.0,6.0,1.0,6.0,135.0
2,2018,24,TEN,RB,22.0,16.0,12.0,215.0,1059.0,12.0,...,6.2,83.3,5.5,230.0,5.0,1158.0,12.0,1.0,7.0,202.8
3,2019,25,TEN,RB,22.0,15.0,15.0,303.0,1540.0,16.0,...,13.7,75.0,8.6,321.0,5.4,1746.0,18.0,5.0,13.0,300.6
4,2020,26,TEN,RB,22.0,16.0,16.0,378.0,2027.0,17.0,...,7.1,61.3,3.7,397.0,5.4,2141.0,17.0,3.0,17.0,335.1
